In [1]:
import os
import sys

sys.path.insert(1, os.path.join(sys.path[0], ".."))

import numpy as np
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import torch

import data_utils.utils as data_utils
import inference.utils as inference_utils
import BigGAN_PyTorch.utils as biggan_utils
from data_utils.datasets_common import pil_loader
import torchvision.transforms as transforms
import time
from glob import glob

Faiss library not found!


In [2]:
import torch
import torchvision
from glob import glob
import utils
import os
from PIL import Image
from data_utils.resnet import resnet50

In [3]:
def get_data(root_path, model, resolution, which_dataset, visualize_instance_images):
    data_path = os.path.join(root_path, "stored_instances")
    if model == "cc_icgan":
        feature_extractor = "classification"
    else:
        feature_extractor = "selfsupervised"
    filename = "%s_res%i_rn50_%s_kmeans_k1000_instance_features.npy" % (
        which_dataset,
        resolution,
        feature_extractor,
    )
    # Load conditioning instances from files
    data = np.load(os.path.join(data_path, filename), allow_pickle=True).item()

    transform_list = None
    if visualize_instance_images:
        # Transformation used for ImageNet images.
        transform_list = transforms.Compose(
            [data_utils.CenterCropLongEdge(), transforms.Resize(resolution)]
        )
    return data, transform_list


def get_model(exp_name, root_path, backbone, device="cuda"):
    parser = biggan_utils.prepare_parser()
    parser = biggan_utils.add_sample_parser(parser)
    parser = inference_utils.add_backbone_parser(parser)

    args = ["--experiment_name", exp_name]
    args += ["--base_root", root_path]
    args += ["--model_backbone", backbone]

    config = vars(parser.parse_args(args=args))

    # Load model and overwrite configuration parameters if stored in the model
    config = biggan_utils.update_config_roots(config, change_weight_folder=False)
    generator, config = inference_utils.load_model_inference(config, device=device)
    biggan_utils.count_parameters(generator)
    generator.eval()

    return generator


def get_conditionings(test_config, generator, data):
    # Obtain noise vectors
    z = torch.empty(
        5 * 5,
        generator.z_dim if "biggan" == "stylegan2" else generator.dim_z,
    ).normal_(mean=0, std=1.0)

    # Subsampling some instances from the 1000 k-means centers file
    if 5 > 1:
        total_idxs = np.random.choice(
            range(1000), 5, replace=False
        )

    # Obtain features, labels and ground truth image paths
    all_feats, all_img_paths, all_labels = [], [], []
    for counter in range(5):
        # Index in 1000 k-means centers file
        if None is not None:
            idx = None
        else:
            idx = total_idxs[counter]
        # Image paths to visualize ground-truth instance
        if False:
            all_img_paths.append(data["image_path"][idx])
        # Instance features
        all_feats.append(
            torch.FloatTensor(data["instance_features"][idx : idx + 1]).repeat(
                5, 1
            )
        )
        # Obtain labels
        if None is not None:
            # Swap label for a manually specified one
            label_int = None
        else:
            # Use the label associated to the instance feature
            label_int = int(data["labels"][idx])
        # Format labels according to the backbone
        labels = None
        if "biggan" == "stylegan2":
            dim_labels = 1000
            labels = torch.eye(dim_labels)[torch.LongTensor([label_int])].repeat(
                5, 1
            )
        else:
            if "icgan" == "cc_icgan":
                labels = torch.LongTensor([label_int]).repeat(
                    5
                )
        all_labels.append(labels)
    # Concatenate all conditionings
    all_feats = torch.cat(all_feats)
    if all_labels[0] is not None:
        all_labels = torch.cat(all_labels)
    else:
        all_labels = None
    return z, all_feats, all_labels, all_img_paths

In [4]:
net = utils.load_pretrained_feature_extractor(
        '/Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path/swav_800ep_pretrain.pth.tar',
        "selfsupervised",
        "resnet50",
    )
net = torch.nn.DataParallel(net)
net.to("cuda")
net.eval()

using resnet50 to extract features
Loading pretrained weights from:  /Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path/swav_800ep_pretrain.pth.tar
key  projection_head.0.weight  not in dict
key  projection_head.0.bias  not in dict
key  projection_head.1.weight  not in dict
key  projection_head.1.bias  not in dict
key  projection_head.1.running_mean  not in dict
key  projection_head.1.running_var  not in dict
key  projection_head.1.num_batches_tracked  not in dict
key  projection_head.3.weight  not in dict
key  projection_head.3.bias  not in dict
key  prototypes.weight  not in dict
Network key  fc.weight  not in dict to load
Network key  fc.bias  not in dict to load


DataParallel(
  (module): ResNet_mine(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
 

In [5]:
# net = utils.load_pretrained_feature_extractor(
#         "/Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path/icgan_biggan_imagenet_res256/state_dict_best0.pth",
#         "classification",
#         "resnet50",
#     )
# net  = net.eval()

In [6]:
device = "cuda"
exp_name = "%s_%s_%s_res%i%s" % (
        "icgan",
        "biggan",
        "imagenet",
        256,
        "",
    )
generator = get_model(
        exp_name, "/Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path", "biggan", device=device
    )

generator = torch.nn.DataParallel(generator)
generator.to("cuda")
generator.eval()

Pegging all root folders to base root /Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path
For name best  best0  we have an FID:  22.453704833984375
Checkpoint with name  best1  not in folder.
Final name selected is  best0
Loading best0 weights from /Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path/icgan_biggan_imagenet_res256...
Experiment name is icgan_biggan_imagenet_res256
Adding attention layer in G at resolution 64
Param count for Gs initialized parameters: 90014147
Loading weights...
Loading best0 weights from /Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path/icgan_biggan_imagenet_res256...
Putting G in eval mode..
Number of parameters: 90014340


DataParallel(
  (module): Generator(
    (activation): ReLU()
    (shared): Embedding(1000, 128)
    (shared_feat): SNLinear(in_features=2048, out_features=512, bias=True)
    (linear): SNLinear(in_features=17, out_features=24576, bias=True)
    (blocks): ModuleList(
      (0): ModuleList(
        (0): GBlock(
          (activation): ReLU()
          (conv1): SNConv2d(1536, 1536, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): SNConv2d(1536, 1536, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv_sc): SNConv2d(1536, 1536, kernel_size=(1, 1), stride=(1, 1))
          (bn1): ccbn(
            out: 1536, in: 529, cross_replica=False
            (gain): SNLinear(in_features=529, out_features=1536, bias=False)
            (bias): SNLinear(in_features=529, out_features=1536, bias=False)
          )
          (bn2): ccbn(
            out: 1536, in: 529, cross_replica=False
            (gain): SNLinear(in_features=529, out_features=1536, bias=False)
     

In [ ]:
# feat_map0 = torch.load("/Work2/Watch_This/ICGAN/ic_gan/data_utils/test_images/0_feat_map.pth").unsqueeze(0)
# img0 = Image.open("/Work2/Watch_This/ICGAN/ic_gan/data_utils/test_images/0_img.JPEG") 
# img0

In [ ]:
# z = torch.normal(0.0, 0.0, (1, 119)) 

# out_img0 = generator(
#             z[0:1].to(device), None, feat_map0.to(device)
#         )
# out_img0 = torch.clamp((out_img0 * 0.5 + 0.5), 0, 1)
# torchvision.transforms.functional.to_pil_image(out_img0[0])

In [ ]:
# feat_map1 = torch.load("/Work2/Watch_This/ICGAN/ic_gan/data_utils/test_images/1_feat_map.pth").unsqueeze(0)
# img1 = Image.open("/Work2/Watch_This/ICGAN/ic_gan/data_utils/test_images/1_img.JPEG") 
# img1

In [ ]:
# z = torch.normal(0.0, 0.0, (1, 119)) 

# out_img1 = generator(
#             z[0:1].to(device), None, feat_map1.to(device)
#         )
# out_img1 = torch.clamp((out_img1 * 0.5 + 0.5), 0, 1)
# torchvision.transforms.functional.to_pil_image(out_img1[0])

In [3]:
files = glob("/Work1/imagenet/val/*")

In [4]:
for file in files:
    os.makedirs(file.replace("imagenet", "ICGAN"), exist_ok=True)

In [5]:
files = glob("/Work1/imagenet/train/*/*")

In [6]:
for file in files:
    os.makedirs(file.replace("imagenet", "ICGAN").replace(".JPEG", ""), exist_ok=True)

In [19]:
dataset = utils.get_dataset_images(
        256,
        data_path="/Work1/imagenet",
        longtail=False,
        split="train",
        test_part=False,
        which_dataset="imagenet",
        instance_json="",
        stuff_json="",
    )

kwargs = {
    "num_workers": 32,
    "pin_memory": True,
    "drop_last": False,
}

train_loader = utils.get_dataloader(
    dataset, 1, shuffle=False, **kwargs
)

Generating  Index file imagenet_imgs.npz...


100%|█████████████████████████████████████████████| 2/2 [00:03<00:00,  1.76s/it]


Dropping last batch?  False
Loader workers? {'num_workers': 32, 'pin_memory': True, 'drop_last': False}  with shuffle? False


In [20]:
norm_mean = torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).cuda()
norm_std = torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).cuda()

In [21]:
total_params1 = sum(p.numel() for p in generator.parameters())
total_params2 = sum(p.numel() for p in net.parameters())
total_params = total_params1+total_params2

In [22]:
print(f"Number of parameters: {total_params}")

Number of parameters: 115571372


In [31]:
class GaussianBlur(object):
    def __init__(self, p):
        self.p = p

    def __call__(self, img):
        sigma = np.random.rand() * 1.9 + 0.1
        return img.filter(ImageFilter.GaussianBlur(sigma))

In [112]:
from torchvision.transforms import InterpolationMode
crop = torchvision.transforms.RandomResizedCrop(224, interpolation=InterpolationMode.BICUBIC)
color_jitter = torchvision.transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)
blur = torchvision.transforms.GaussianBlur(3, sigma=(0.1, 2.0))

In [125]:
def Average(lst):
    return sum(lst) / len(lst)

In [139]:
time_list = []
for i, (x, _, image_id, path) in enumerate(train_loader):
    with torch.no_grad():
        x_tf = x.cuda()
        x_tf = x_tf * 0.5 + 0.5
        x_tf = (x_tf - norm_mean) / norm_std
        x_tf = torch.nn.functional.upsample(x_tf, 224, mode="bicubic")

        start_time = time.time()
        # x = crop(x)
        x_feat, _ = net(x_tf)
        x_feat /= torch.linalg.norm(x_feat, dim=1, keepdims=True)

                
        z = torch.normal(0.0, 1.0, (x_tf.shape[0], 119)).cuda() 

        x_img = generator(z, None, x_feat)
        end_time = time.time()

        time_list.append(end_time-start_time)
        if i > 100000:
            break
        # x = x.cpu()
#         x_feat = x_feat.cpu()
#         x_img = x_img.cpu()
        
#         for j in range(x_tf.shape[0]):
#             folder = path[j].replace("imagenet", "ICGAN").replace(".JPEG", "")
#             file_name = folder.split("/")[-1]
#             test = folder + "/" + file_name + ".JPEG"
#             real_img = torch.clamp((x[j] * 0.5 + 0.5), 0, 1)
#             torchvision.transforms.functional.to_pil_image(real_img).save(folder + "/" + file_name + "_R.JPEG")
#             gen_img = torch.clamp((x_img[j] * 0.5 + 0.5), 0, 1)
#             torchvision.transforms.functional.to_pil_image(gen_img).save(folder + "/" + file_name + "_G.JPEG")
# #             torch.save(x_feat[j], folder + "/" + file_name + "_F.pt")
        
#     print(str((i/len(train_loader))*100.0) + "% Done, Epoch: " + str(i) + "/" + str(len(train_loader))) 

In [140]:
Average(time_list[1:])

0.019820995125105052

In [132]:
# input_img = torch.clamp((x[0] * 0.5 + 0.5), 0, 1)
# torchvision.transforms.functional.to_pil_image(input_img)

In [ ]:
# x_img = torch.clamp((x_img * 0.5 + 0.5), 0, 1)
# torchvision.transforms.functional.to_pil_image(x_img[0])

In [ ]:
# feat_map0.shape

In [ ]:
# import torch

In [ ]:
# feat_map0 = torch.load("/Work2/Watch_This/ICGAN/ic_gan/data_utils/test_images/0_feat_map.pth").unsqueeze(0)
# img0 = Image.open("/Work2/Watch_This/ICGAN/ic_gan/data_utils/test_images/0_img.JPEG") 
# tensor0 = torchvision.transforms.functional.to_tensor(img0).unsqueeze(0).cuda()
# tensor0 = (tensor0 - norm_mean.cuda()) / norm_std.cuda()
# tensor0 = torch.nn.functional.upsample(tensor0, 224, mode="bicubic")
# img0

In [ ]:
# torchvision.transforms.functional.to_pil_image(tensor0[0])

In [ ]:
# z.shape, x_feat.shape

In [ ]:
# x_feat0, _ = net(tensor0.cuda())
        
# z = torch.normal(0.0, 1.0, (1, 119)).cuda() 

# x_img = generator(z.cuda(), None, x_feat[0:1].cuda())

# x_img = torch.clamp((x_img * 0.5 + 0.5), 0, 1)
# torchvision.transforms.functional.to_pil_image(x_img[0])

In [ ]:
# input_img = torch.clamp((x * 0.5 + 0.5), 0, 1)
# torchvision.transforms.functional.to_pil_image(input_img[0])

In [ ]:
# out_img = torch.clamp((x_img * 0.5 + 0.5), 0, 1)
# torchvision.transforms.functional.to_pil_image(out_img[0])

In [ ]:
# if args["which_dataset"] == "imagenet":
#     dataset_name_prefix = "ILSVRC"
#     im_prefix = "IN"
# elif args["which_dataset"] == "coco":
#     dataset_name_prefix = "COCO"
#     im_prefix = "COCO"
# else:
#     dataset_name_prefix = args["which_dataset"]
#     im_prefix = args["which_dataset"]
# # HDF5 filename
# filename = os.path.join(
#     args["data_root"],
#     "%s%s_feats_%s_%s.hdf5"
#     % (
#         dataset_name_prefix,
#         args["resolution"],
#         args["feature_extractor"],
#         args["backbone_feature_extractor"],
#     ),
# )
# # Load features
# print("Loading features %s..." % (filename))
# with h5.File(filename, "r") as f:
#     features = f["feats"][:]
# features = np.array(features)
# # Normalize features
# features /= np.linalg.norm(features, axis=1, keepdims=True)

# feat_dim = 2048
# # k-means
# print("Training k-means with %i centers..." % (args["kmeans_subsampled"]))
# kmeans = faiss.Kmeans(
#     feat_dim,
#     args["kmeans_subsampled"],
#     niter=100,
#     verbose=True,
#     gpu=args["gpu"],
#     min_points_per_centroid=200,
#     spherical=False,
# )
# kmeans.train(features.astype(np.float32))

# # Find closest instances to each k-means cluster
# print("Finding closest instances to centers...")
# index = faiss.IndexFlatL2(feat_dim)
# index.add(features.astype(np.float32))
# D, closest_sample = index.search(kmeans.centroids, 1)

# net_str = (
#     "rn50"
#     if args["backbone_feature_extractor"]
#     else args["backbone_feature_extractor"]
# )
# stored_filename = "%s_res%i_%s_%s_kmeans_k%i" % (
#     im_prefix,
#     args["resolution"],
#     net_str,
#     args["feature_extractor"],
#     args["kmeans_subsampled"],
# )
# np.save(
#     os.path.join(args["data_root"], stored_filename),
#     {"center_examples": closest_sample},
# )
# print(
#     "Instance indexes resulting from a subsampling based on k-means have been saved in file %s!"
#     % (stored_filename)
# )

In [9]:
files = glob("/Work1/imagenet/*/*/*")

In [19]:
for ind, file in enumerate(files):
    if ind < 940000: continue
    im = Image.open(file).convert("RGB") 
    folder = file.replace("imagenet", "ICGAN").replace(".JPEG", "")
    file_name = file.split("/")[-1].replace(".JPEG", "")
    im.save(folder + "/" + file_name + "_R.JPEG")
    if ind % 10000 == 0:
        print(str(ind) + "/" + str(len(files)) + " " + str((ind/len(files))*100) + "%")

940000/1331167 70.61473128465474%
950000/1331167 71.36595183023617%
960000/1331167 72.1171723758176%
970000/1331167 72.86839292139904%
980000/1331167 73.61961346698047%
990000/1331167 74.37083401256191%
1000000/1331167 75.12205455814335%
1010000/1331167 75.87327510372478%
1020000/1331167 76.62449564930621%
1030000/1331167 77.37571619488764%
1040000/1331167 78.12693674046908%
1050000/1331167 78.87815728605051%
1060000/1331167 79.62937783163194%
1070000/1331167 80.38059837721337%
1080000/1331167 81.1318189227948%
1090000/1331167 81.88303946837624%
1100000/1331167 82.63426001395769%
1110000/1331167 83.38548055953912%
1120000/1331167 84.13670110512055%
1130000/1331167 84.88792165070198%
1140000/1331167 85.63914219628342%
1150000/1331167 86.39036274186485%
1160000/1331167 87.14158328744628%
1170000/1331167 87.89280383302771%
1180000/1331167 88.64402437860915%
1190000/1331167 89.39524492419058%
1200000/1331167 90.14646546977201%
1210000/1331167 90.89768601535344%
1220000/1331167 91.648906560